In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install cufflinks

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install plotly


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import numpy as np
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline 
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
cf.go_offline();
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

pd.set_option("display.max_columns", None)

In [8]:
df = pd.read_csv('amazon.csv')

In [9]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [10]:
df = df.sort_values("wilson_lower_bound", ascending = False)
df.drop('Unnamed: 0', inplace = True, axis=1)
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [11]:
import pandas as pd

def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    ratio_dict = ratio_.to_dict()  # Convert Series to a dictionary
    missing_df = pd.concat([n_miss, pd.Series(ratio_dict)], axis=1, keys=['Missing values', 'Ratio'])
    return missing_df

def check_dataframe(df, head=5, tail=5):
    
    print("SHAPE".center(82,'~'))
    print('Rows: {}'.format(df.shape[0]))
    print('columns: {}'.format(df.shape[1]))
    print("TYPES".center(82,'~'))
    print(df.dtypes)
    print("".center(82, '~'))
    print(missing_values_analysis(df))
    print('DUPLICATED VALUES'.center(83,'~'))
    print(df.duplicated().sum())
    print("QUANTILES".center(82, '~'))
    # Select only numeric columns before calculating quantiles
    numeric_columns = df.select_dtypes(include=[np.number])
    print(numeric_columns.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
                         
check_dataframe(df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows: 4915
columns: 11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing values     Ratio
reviewerName               1  0.020346
reviewText                 1  0.020346
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~QUANTILES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       0.00  0.05   0.50        0.95       0.99         1.00

In [12]:
def check_class(dataframe):
    nunique_df = pd.DataFrame({'Variable': dataframe.columns,
                               'Classes': [dataframe[col].nunique() for col in dataframe.columns]})
    nunique_df = nunique_df.sort_values('Classes', ascending=False)
    nunique_df = nunique_df.reset_index(drop=True)
    return nunique_df

check_class(df)

,Variable,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [13]:
constraints = ['#B34D22','#EBE00C','#1FEB0C','#0C92EB','#EB0CD5']

def categorical_variable_summary(df, column_name):
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Countplot', 'Percentage'),
                       specs=[[{"type": "xy"}, {'type': 'domain'}]])
    
    fig.add_trace(go.Bar( y= df[column_name].value_counts().values.tolist(),
                        x= [str(i) for i in df[column_name].value_counts().index],
                        text = df[column_name].value_counts().values.tolist(),
                        textfont = dict(size=14),
                         name =  column_name,
                         textposition = 'auto',
                         showlegend = False,
                         marker = dict(color = constraints,line=dict(color = '#DBE6EC',
                                                                    width = 1))),
                  row = 1, col = 1)
    
    fig.add_trace(go.Pie(labels= df[column_name].value_counts().keys(),
                        values= df[column_name].value_counts().values,    
                        textfont = dict(size=18),
                        textposition =  'auto',
                        showlegend = False,
                        name = column_name,
                        marker = dict(colors = constraints)),
                  row = 1, col = 2)
    
    fig.update_layout(title={'text': column_name,
                                   'y':0.9,
                                   'x':0.5,
                                   'xanchor':'center',
                                   'yanchor': 'top'},
                             template = 'plotly_white')
                             
    iplot(fig)
    
    

In [14]:
categorical_variable_summary(df, 'overall')

In [15]:
# To clean data
df.reviewText.head()

2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [16]:
review_example = df.reviewText[2031]
review_example

'[[ UPDATE - 6/19/2014 ]]So my lovely wife bought me a Samsung Galaxy Tab 4 for Father\'s Day and I\'ve been loving it ever since.  Just as other with Samsung products, the Galaxy Tab 4 has the ability to add a microSD card to expand the memory on the device.  Since it\'s been over a year, I decided to do some more research to see if SanDisk offered anything new.  As of 6/19/2014, their product lineup for microSD cards from worst to best (performance-wise) are the as follows:SanDiskSanDisk UltraSanDisk Ultra PLUSSanDisk ExtremeSanDisk Extreme PLUSSanDisk Extreme PRONow, the difference between all of these cards are simply the speed in which you can read/write data to the card.  Yes, the published rating of most all these cards (except the SanDisk regular) are Class 10/UHS-I but that\'s just a rating... Actual real world performance does get better with each model, but with faster cards come more expensive prices.  Since Amazon doesn\'t carry the Ultra PLUS model of microSD card, I had 

In [17]:
# to remove punctuatuins, numbers and spaces from data
review_example = re.sub("[^a-zA-Z]",'',review_example)
review_example

'UPDATESomylovelywifeboughtmeaSamsungGalaxyTabforFathersDayandIvebeenlovingiteversinceJustasotherwithSamsungproductstheGalaxyTabhastheabilitytoaddamicroSDcardtoexpandthememoryonthedeviceSinceitsbeenoverayearIdecidedtodosomemoreresearchtoseeifSanDiskofferedanythingnewAsoftheirproductlineupformicroSDcardsfromworsttobestperformancewisearetheasfollowsSanDiskSanDiskUltraSanDiskUltraPLUSSanDiskExtremeSanDiskExtremePLUSSanDiskExtremePRONowthedifferencebetweenallofthesecardsaresimplythespeedinwhichyoucanreadwritedatatothecardYesthepublishedratingofmostallthesecardsexcepttheSanDiskregularareClassUHSIbutthatsjustaratingActualrealworldperformancedoesgetbetterwitheachmodelbutwithfastercardscomemoreexpensivepricesSinceAmazondoesntcarrytheUltraPLUSmodelofmicroSDcardIhadtododirectcomparisonsbetweentheSanDiskUltraExtremeandExtremePLUSAsmentionedinmyearlierreviewIpurchasedtheSanDiskUltraformyGalaxySMyquestionwasdidIwanttopayovermoreforacardthatisfasterthantheoneIalreadyownedOrIcouldpayalmostdoubletoget

In [18]:
# Make all text in lowecase
# Assuming review_example is a list of words
review_example = " ".join(review_example)  # Convert the list to a single string
review_example = review_example.lower().split()  # Convert to lowercase and split into words

In [19]:
review_example

['u',
 'p',
 'd',
 'a',
 't',
 'e',
 's',
 'o',
 'm',
 'y',
 'l',
 'o',
 'v',
 'e',
 'l',
 'y',
 'w',
 'i',
 'f',
 'e',
 'b',
 'o',
 'u',
 'g',
 'h',
 't',
 'm',
 'e',
 'a',
 's',
 'a',
 'm',
 's',
 'u',
 'n',
 'g',
 'g',
 'a',
 'l',
 'a',
 'x',
 'y',
 't',
 'a',
 'b',
 'f',
 'o',
 'r',
 'f',
 'a',
 't',
 'h',
 'e',
 'r',
 's',
 'd',
 'a',
 'y',
 'a',
 'n',
 'd',
 'i',
 'v',
 'e',
 'b',
 'e',
 'e',
 'n',
 'l',
 'o',
 'v',
 'i',
 'n',
 'g',
 'i',
 't',
 'e',
 'v',
 'e',
 'r',
 's',
 'i',
 'n',
 'c',
 'e',
 'j',
 'u',
 's',
 't',
 'a',
 's',
 'o',
 't',
 'h',
 'e',
 'r',
 'w',
 'i',
 't',
 'h',
 's',
 'a',
 'm',
 's',
 'u',
 'n',
 'g',
 'p',
 'r',
 'o',
 'd',
 'u',
 'c',
 't',
 's',
 't',
 'h',
 'e',
 'g',
 'a',
 'l',
 'a',
 'x',
 'y',
 't',
 'a',
 'b',
 'h',
 'a',
 's',
 't',
 'h',
 'e',
 'a',
 'b',
 'i',
 'l',
 'i',
 't',
 'y',
 't',
 'o',
 'a',
 'd',
 'd',
 'a',
 'm',
 'i',
 'c',
 'r',
 'o',
 's',
 'd',
 'c',
 'a',
 'r',
 'd',
 't',
 'o',
 'e',
 'x',
 'p',
 'a',
 'n',
 'd',
 't',
 'h'

In [20]:
rt = lambda x: re.sub("[^a-zA-Z]", ' ',str(x))
df["reviewText"] = df["reviewText"].map(rt)
df["reviewText"] = df["reviewText"].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [21]:
pip install vaderSentiment


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# TextBlob will return polarity and subjectivity
# polarity indicates your mood i.e - positive, neutraol or negative
# It's value is betwn 0 and 1
# The more closer to zero, the more negative

df[['polarity', 'subjectivity']] = df['reviewText'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

analyzer = SentimentIntensityAnalyzer()

# Perform sentiment analysis using VADER for each review in the 'reviewText' column
for index, row in df.iterrows():
    score = analyzer.polarity_scores(row['reviewText'])
    print(score)
    
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    
    if neg > pos:
        df.loc[index, 'sentiment'] = "Negative"
    elif pos > neg:
        df.loc[index, ' sentiment'] = "Positive"
    else:
        df.loc[index, 'sentiment'] = "Neutral"

{'neg': 0.031, 'neu': 0.859, 'pos': 0.111, 'compound': 0.9981}
{'neg': 0.023, 'neu': 0.89, 'pos': 0.087, 'compound': 0.991}
{'neg': 0.046, 'neu': 0.807, 'pos': 0.147, 'compound': 0.9995}
{'neg': 0.038, 'neu': 0.885, 'pos': 0.076, 'compound': 0.8934}
{'neg': 0.029, 'neu': 0.839, 'pos': 0.132, 'compound': 0.9888}
{'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'compound': 0.7184}
{'neg': 0.052, 'neu': 0.827, 'pos': 0.121, 'compound': 0.8393}
{'neg': 0.107, 'neu': 0.811, 'pos': 0.082, 'compound': -0.2067}
{'neg': 0.024, 'neu': 0.884, 'pos': 0.091, 'compound': 0.7514}
{'neg': 0.042, 'neu': 0.88, 'pos': 0.078, 'compound': 0.9809}
{'neg': 0.0, 'neu': 0.884, 'pos': 0.116, 'compound': 0.9601}
{'neg': 0.026, 'neu': 0.875, 'pos': 0.099, 'compound': 0.9605}
{'neg': 0.0, 'neu': 0.936, 'pos': 0.064, 'compound': 0.5719}
{'neg': 0.043, 'neu': 0.805, 'pos': 0.152, 'compound': 0.9833}
{'neg': 0.0, 'neu': 0.96, 'pos': 0.04, 'compound': 0.4215}
{'neg': 0.054, 'neu': 0.856, 'pos': 0.09, 'compound': 0.3597}
{'neg

In [23]:
df[df['sentiment']=='Positive'].sort_values("wilson_lower_bound",
                                           ascending = False).head(5)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment,sentiment


In [24]:
categorical_variable_summary(df,'sentiment')